In [29]:
import sys
sys.path.append("..")  # remonte d'un niveau vers le dossier parent
from utils.utils import connect_to_db, insert_data_to_db, mapping_colonnes, fetch_data_from_db
engine = connect_to_db()
df = fetch_data_from_db(table_name='accidents_raw', engine=engine, schema='bronze')
df.columns

✅ Connexion à la base PostgreSQL réussie !
OK: "bronze"."accidents_raw"


Index(['Identifiant de l'accident', 'Date et heure', 'Commune', 'Année',
       'Mois', 'Jour', 'Heure minute', 'Lumière', 'Localisation',
       'Intersection', 'Conditions atmosphériques', 'Collision', 'Département',
       'Code commune', 'Code Insee', 'Adresse', 'Latitude', 'Longitude',
       'Code Postal', 'Numéro', 'Coordonnées', 'PR', 'Surface', 'V1',
       'Circulation', 'Voie réservée', 'Env1', 'Voie',
       'Largeur de la chaussée', 'V2', 'Largeur terre plein central',
       'Nombre de voies', 'Catégorie route', 'PR1', 'Plan', 'Profil',
       'Infrastructure', 'Situation', 'Année de naissance', 'Sexe',
       'Action piéton', 'Gravité', 'Existence équipement de sécurité',
       'Utilisation équipement de sécurité', 'Localisation du piéton',
       'Identifiant véhicule', 'Place', 'Catégorie d'usager',
       'Piéton seul ou non', 'Motif trajet', 'Point de choc', 'Manœuvre',
       'Sens', 'Obstacle mobile heurté', 'Obstacle fixe heurté',
       'Catégorie véhicule', 'No

In [30]:
mapping_colonnes = {
    # --- Caractéristiques de l'accident (12) ---
    "Identifiant de l'accident": "num_acc",
    "Date et heure": "datetime",
    "Année": "an",
    "Mois": "mois",
    "Jour": "jour",
    "Heure minute": "hrmn",
    "Lumière": "lum",
    "Localisation": "agg",
    "Intersection": "int",
    "Conditions atmosphériques": "atm",
    "Collision": "col",
    "Situation": "situ",

    # --- Lieu (30) ---
    "Département": "dep",
    "Code commune": "com",
    "Code Insee": "insee",
    "Adresse": "adr",
    "Latitude": "lat",
    "Longitude": "long",
    "Code Postal": "code_postal",
    "Numéro": "num",
    "Coordonnées": "coordonnees",
    "PR": "pr",
    "Surface": "surf",
    "V1": "v1",
    "Circulation": "circ",
    "Voie réservée": "vosp",
    "Env1": "env1",
    "Voie": "voie",
    "Largeur de la chaussée": "larrout",
    "V2": "v2",
    "Largeur terre plein central": "lartpc",
    "Nombre de voies": "nbv",
    "Catégorie route": "catr",
    "PR1": "pr1",
    "Plan": "plan",
    "Profil": "prof",
    "Infrastructure": "infra",
    "Gps": "gps",
    "date": "date", 
    "year_georef": "year_georef",
    "Commune": "nom_com", 
    "Nom Officiel Commune": "com_name",

    # --- Usager (11) ---
    "Année de naissance": "an_nais",
    "Sexe": "sexe",
    "Action piéton": "actp",
    "Gravité": "grav",
    "Existence équipement de sécurité": "secu",
    "Utilisation équipement de sécurité": "secu_utl",
    "Localisation du piéton": "locp",
    "Place": "place",
    "Catégorie d'usager": "catu",
    "Piéton seul ou non": "etatp",
    "Motif trajet": "trajet",

    # --- Véhicule (8) ---
    "Identifiant véhicule": "num_veh",
    "Point de choc": "choc",
    "Manœuvre": "manv",
    "Sens": "senc",
    "Obstacle mobile heurté": "obsm",
    "Obstacle fixe heurté": "obs",
    "Catégorie véhicule": "catv",
    "Nombre d'occupants": "occutc",

    # --- Noms et Codes Officiels (8) ---
    "Code Officiel Département": "dep_code", 
    "Nom Officiel Département": "dep_name",
    "Code Officiel EPCI": "epci_code",       
    "Nom Officiel EPCI": "epci_name",
    "Code Officiel Région": "reg_code",       
    "Nom Officiel Région": "reg_name",
    "Nom Officiel Commune / Arrondissement Municipal": "com_arm_name",
    "Code Officiel Commune": "code_com"         
}

len(mapping_colonnes)

69

In [31]:
df = df.rename(columns=mapping_colonnes)


# --- Colonnes à conserver selon la table silver.region ---
colonnes_accident = ["num_acc", "com", "adr","datetime","year_georef"]

# --- Vérifier que les colonnes existent et les filtrer ---
colonnes_disponibles = [col for col in colonnes_accident if col in df.columns]

if not colonnes_disponibles:
    raise ValueError('⚠️ Aucune des colonnes ["num_acc", "com", "adr","datetime","year_georef"] n\'existe dans accidents_raw.')


df_accident = df[colonnes_disponibles]

print("✅ Données filtrées pour la table 'silver.accident' :")


✅ Données filtrées pour la table 'silver.accident' :


In [32]:
# Supprimer les doublons sur epci_code (car c’est une clé unique)
df_accident = df_accident.drop_duplicates(subset="num_acc", keep="first")
#supprimer les lignes vide
df_accident = df_accident.dropna(how="all")

df_accident = df_accident.sort_values(by="num_acc", ascending=True)
print(df_accident.head(10))

             num_acc  com                     adr                   datetime  \
302293  201200000001  011   33 Rue Roger Salengro  2012-03-16T19:30:00+01:00   
250226  201200000002  011            route de DON  2012-09-01T21:45:00+02:00   
353405  201200000003  670      rue Anatole France  2012-11-20T18:15:00+01:00   
50968   201200000004  051          Rue d'ESTAIRES  2012-01-07T19:15:00+01:00   
201995  201200000005  524  Rue du Général Leclerc  2012-01-17T20:00:00+01:00   
265458  201200000006  051      34 rue de la Marne  2012-01-22T18:00:00+01:00   
213542  201200000007  051          13 RUE DE LENS  2012-04-14T16:45:00+02:00   
338147  201200000008  524        533 Rue Gambetta  2012-07-13T16:00:00+02:00   
25030   201200000009  550           Rte Nationale  2012-08-10T17:00:00+02:00   
75352   201200000010  388            Rue du Faulx  2012-09-14T04:45:00+02:00   

       year_georef  
302293        2015  
250226        2015  
353405        2015  
50968         2015  
201995        

In [33]:
print("🔍 Valeurs manquantes avant nettoyage :")
print(df_accident.isna().sum())
# Supprimer les lignes sans num_acc
df_accident = df_accident.dropna(subset=["num_acc"])
# Remplacer les valeurs manquantes 
df_accident["adr"] = df_accident["adr"].fillna("Inconnu")
df_accident["datetime"] = df_accident["datetime"].fillna("Inconnu")
df_accident["year_georef"] = df_accident["year_georef"].fillna("Inconnu")
df_accident["com"] = df_accident["com"].fillna("Inconnu")

🔍 Valeurs manquantes avant nettoyage :
num_acc            0
com                0
adr            49256
datetime           0
year_georef        0
dtype: int64


In [34]:
df_accident = df_accident.loc[:, ~df_accident.columns.duplicated()]
print("✅ Aperçu des données nettoyées :")
print(df_accident.head(10))
print("🔍 Nombre de valeurs nulles par colonne :")
print(df_accident.isna().sum())

✅ Aperçu des données nettoyées :
             num_acc  com                     adr                   datetime  \
302293  201200000001  011   33 Rue Roger Salengro  2012-03-16T19:30:00+01:00   
250226  201200000002  011            route de DON  2012-09-01T21:45:00+02:00   
353405  201200000003  670      rue Anatole France  2012-11-20T18:15:00+01:00   
50968   201200000004  051          Rue d'ESTAIRES  2012-01-07T19:15:00+01:00   
201995  201200000005  524  Rue du Général Leclerc  2012-01-17T20:00:00+01:00   
265458  201200000006  051      34 rue de la Marne  2012-01-22T18:00:00+01:00   
213542  201200000007  051          13 RUE DE LENS  2012-04-14T16:45:00+02:00   
338147  201200000008  524        533 Rue Gambetta  2012-07-13T16:00:00+02:00   
25030   201200000009  550           Rte Nationale  2012-08-10T17:00:00+02:00   
75352   201200000010  388            Rue du Faulx  2012-09-14T04:45:00+02:00   

       year_georef  
302293        2015  
250226        2015  
353405        2015  
50

In [35]:
insert_data_to_db(df_accident, table_name='silver.accident', engine=engine)


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "accident" violates foreign key constraint "fk_accident_commune"
DETAIL:  Key (com)=(011) is not present in table "commune".

[SQL: INSERT INTO silver.accident (num_acc,com,adr,datetime,year_georef) VALUES (%(num_acc)s,%(com)s,%(adr)s,%(datetime)s,%(year_georef)s)]
[parameters: [{'num_acc': '201200000001', 'com': '011', 'adr': '33 Rue Roger Salengro', 'datetime': '2012-03-16T19:30:00+01:00', 'year_georef': '2015'}, {'num_acc': '201200000002', 'com': '011', 'adr': 'route de DON', 'datetime': '2012-09-01T21:45:00+02:00', 'year_georef': '2015'}, {'num_acc': '201200000003', 'com': '670', 'adr': 'rue Anatole France', 'datetime': '2012-11-20T18:15:00+01:00', 'year_georef': '2015'}, {'num_acc': '201200000004', 'com': '051', 'adr': "Rue d'ESTAIRES", 'datetime': '2012-01-07T19:15:00+01:00', 'year_georef': '2015'}, {'num_acc': '201200000005', 'com': '524', 'adr': 'Rue du Général Leclerc', 'datetime': '2012-01-17T20:0